## PMSOCR本地部署

### 配置paddle环境  


教程位置： https://github.com/PaddlePaddle/PaddleOCR/blob/release/2.6/ppstructure/docs/quickstart.md


#### 安装PaddlePaddle

In [ ]:
# cpu版本
!python -m pip install paddlepaddle -i https://mirror.baidu.com/pypi/simple

#### 安装PaddleOCR whl包

In [ ]:
# 安装 paddleocr，推荐使用2.6版本
!pip3 install "paddleocr>=2.6.0.3" -i https://mirror.baidu.com/pypi/simple

# 安装 图像方向分类依赖包paddleclas（如不需要图像方向分类功能，可跳过）
!pip3 install paddleclas>=2.4.3 -i https://mirror.baidu.com/pypi/simple

#### 克隆项目文件夹

In [1]:
!git clone https://github.com/Strepsiades/Paddle_Service.git

!cd Paddle_Service

^C


Cloning into 'Paddle_Service'...


#### 开启服务

In [ ]:
# 使用flask搭建一个简单的服务器，默认端口8888
!python pmsocr_service.py

# 使用测试请求
!python client.py


### 在Docker中部署

#### 拉取Paddle默认镜像

不建议使用更小的镜像，可能会报错

教程：https://www.paddlepaddle.org.cn/documentation/docs/zh/install/docker/linux-docker.html#anchor-0

In [ ]:
# CPU 版的 PaddlePaddle:
!docker pull registry.baidubce.com/paddlepaddle/paddle:2.4.1

#### 进入交互模式配置环境

In [ ]:
# 首先构建容器, 并进行端口映射
!docker run --name -p 8888:8888 paddle_docker -it -v registry.baidubce.com/paddlepaddle/paddle:2.4.1 /bin/bash

# 进入docker如本地操作

# TODO 写一个服务自启动脚本

# 注意service的主机一定要是0.0.0.0, 不能是localhost

# 注意-it模式如果关掉所有守护程序，docker可能关闭容器。使用-dit可以避免这一点


#### 其他Docker命令

```
# 进入已开启的容器
docker exec -i -t  paddle_docker /bin/bash

# 查看已有容器
docker ps -a

```

### 在服务器上部署Docker

#### 将本地容器导出

```
# 语法
docker export [OPTIONS] CONTAINER

# 举例
docker export -o mysql-`date +%Y%m%d`.tar a404c6c174a2

```

#### 使用ftp工具将镜像
可以使用ftp工具将镜像上传至服务器。注意根据服务器情况，一般使用sftp协议，ftp协议可能会报错

#### 将打包好的tar文件还原为镜像
```
# 语法
docker import [OPTIONS] file|URL|- [REPOSITORY[:TAG]]
# 举例
docker import  my_ubuntu_v3.tar runoob/ubuntu:v4
```
#### 构建Docker容器并开启服务
同本地创建。注意，请使用-dit模式，否则服务将随着终端关闭而关闭

TODO:写一个开机自动运行脚本

#### 在nginx中配置端口映射
具体来说，在etc/nginx/nginx.conf中修改配置文件

在http代码块中，插入
```
server {
	listen       120;  # 端口是自定义的，根据服务器防火墙开启情况
	server_name 81.68.184.250;  #  根据服务器ip或网址
	client_max_body_size 10M;  #  设置上传限制
	index  index.php index.html index.htm;
	
	location /pmsocr/ {   # 请求只有带/pmsocr/的url才会被转发
		proxy_pass http://127.0.0.1:8888;   # 端口和paddle服务的端口对应
		proxy_set_header Host $proxy_host; # 修改转发请求头，让对应端口的应用可以受到真实的请求
	}
}
```

然后重启nginx
```
nginx -s reload
```


